In [1]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta

import torch
import torch_geometric
from tqdm.notebook import tqdm
import copy
from typing import List, Optional, Tuple, Union

from torch import Tensor

from torch_geometric.utils.mask import index_to_mask
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.typing import OptTensor, PairTensor
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.data import HeteroData
from torch_geometric_temporal.nn.hetero.heterogclstm import HeteroGCLSTM
from torch_geometric.nn import SAGEConv, GATv2Conv
import torch.nn.functional as F
from torch_geometric.nn.norm import BatchNorm
from torch.nn import Module, ModuleDict
from torch_geometric.nn import to_hetero
#from torch_geometric.utils import bipartite_subgraph
from ast import literal_eval

from sklearn.manifold import TSNE
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [2]:
print(torch.__version__)

1.13.1+cpu


In [3]:
print(torch_geometric.__version__)

2.2.0


In [4]:
init_date='2004-01-01'
final_date='2021-12-31'
init_date_obj = datetime.strptime(init_date, '%Y-%m-%d')
init_date_obj

datetime.datetime(2004, 1, 1, 0, 0)

In [5]:
d=init_date_obj + relativedelta(months=0)
datetime.strftime(d,'%Y-%m-%d')

'2004-01-01'

In [6]:
tracks_info= pd.read_csv(os.path.join('data', 'generated', '05b_all_tracks_w_feat_and_genre.csv'),
                         converters={"artist_id": literal_eval, 'artist_name': literal_eval, 'genres':literal_eval}, 
                         parse_dates=['release_date'],
                         index_col=0)
#tracks_info= tracks_info[(tracks_info['release_date']>init_date) & (tracks_info['release_date']<=final_date)]
#tracks_info= tracks_info[tracks_info['num_genres']>0]

tracks_info= tracks_info.reset_index(drop=True)
tracks_info['num_artists']= tracks_info['artist_id'].apply(lambda x:len(x))

In [7]:
collab_tracks= tracks_info[tracks_info['num_artists']>1]
collab_tracks

,track_id,release_date,release_date_precision,artist_id,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres,num_genres,genres_str,num_artists
1,4ak7xjvBeBOcJGWFDX9w5n,2019-06-21,day,"[7jVv8c5Fj3E9VhNjxT4snq, 4kYSro6naA4h99UJvo89HB]","[Lil Nas X, Cardi B]",0.706,0.67900,9.0,-5.614,1.0,0.0324,0.13900,0.000070,0.4650,0.657,140.081,"[rap, dance, pop]",3,";rap,dance,pop",2
6,5ZbcYWBQWoCpuVip1fmeMY,2018-11-23,day,"[3CJKkU0XuElRT1z8rEtIYg, 6ytOHdKh4xt4YvF7tz8Zcv]","[Luciano, Kalash Criminel]",0.825,0.62600,4.0,-8.444,0.0,0.3380,0.26500,0.000025,0.1010,0.642,142.995,"[german, hip-hop, rap, french, pop]",5,";german,hip-hop,rap,french,pop",2
9,0fcq51a3gOI6gPvzc0YLsk,2019-03-15,day,"[246dkjvS1zLTtiykXe5h60, 757aE44tKEUQEqRuT6GnE...","[Post Malone, Roddy Ricch, Tyga]",0.797,0.64000,11.0,-6.272,0.0,0.1940,0.14800,0.000000,0.2090,0.486,100.002,"[rap, trap, pop, rap, dance, hip-hop]",6,";rap,trap,pop,rap,dance,hip-hop",3
13,3kf0ERnVuqQs62SIpj4HN8,2018-10-26,day,"[6S3KljEiIOWoLMUyZrkQUc, 5uJw4WCX5nYj4FHky9r1Ug]","[Marnik, SMACK]",0.475,0.91400,0.0,-4.255,0.0,0.0555,0.03400,0.000202,0.2770,0.156,138.007,[],0,;,2
16,3tDqEKKUs6gf8zMvSuLyLA,2020-02-28,day,"[5f7VJjfbwm532GiveGC0ZK, 4O15NlyKLIASxsJ0PrXPfz]","[Lil Baby, Lil Uzi Vert]",0.892,0.48600,11.0,-8.637,0.0,0.3840,0.04800,0.000000,0.1460,0.535,120.998,"[hip-hop, rap, trap, rap, trap]",5,";hip-hop,rap,trap,rap,trap",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1966466,6X2jYZpOrXiDXpOvQGcvsU,2015-02-04,day,"[2EWXgN0xWOnbqJOxa9pWNO, 0Pt3k7cpeOysqGGiw3tQcS]","[SawanoHiroyuki[nZk], mizuki]",0.536,0.92900,8.0,-4.829,1.0,0.0567,0.00542,0.000000,0.2190,0.593,157.998,[anime],1,;anime,2
1966479,4F2faUgJBqsrgA51Lz6dNZ,2022-01-08,day,"[1Uff91EOsvd99rtAupatMP, 1p0J5PXJQMVqk5uVV4T1ja]","[Claude Debussy, Seong-Jin Cho]",0.295,0.00642,1.0,-36.008,0.0,0.0380,0.99500,0.925000,0.0610,0.121,77.235,"[classical, classical, piano]",3,";classical,classical,piano",2
1966484,0h9bMMjrFaNJP5oWMq7xPy,2018-10-19,day,"[1GGaiTLrRiVRnTdJygL1Yq, 6q8DZBLBYqEUdKZl54sqG...","[LOWFi, Jayy Grams, Hayelo, Von Wilda]",0.723,0.72900,0.0,-4.701,1.0,0.2310,0.26500,0.000000,0.1030,0.909,89.970,[hip-hop],1,;hip-hop,4
1966486,7iqZMtyAQrp5CnIteoonl1,1999-01-01,day,"[7rPqFVgJYARiFsLDlN6W6y, 5ReVxzuREWusYE4ovnMbBn]","[Toquinho, Paulinho Nogueira]",0.696,0.26000,2.0,-15.253,0.0,0.0510,0.97400,0.885000,0.1100,0.271,119.838,"[mpb, samba, bossa nova, choro, violao, violao...",6,";mpb,samba,bossa nova,choro,violao,violao clas...",2


In [8]:
collab_tracks[collab_tracks['artist_id'].map(lambda x: ('7jVv8c5Fj3E9VhNjxT4snq' in x) and ('4kYSro6naA4h99UJvo89HB' in x))]

,track_id,release_date,release_date_precision,artist_id,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres,num_genres,genres_str,num_artists
1,4ak7xjvBeBOcJGWFDX9w5n,2019-06-21,day,"[7jVv8c5Fj3E9VhNjxT4snq, 4kYSro6naA4h99UJvo89HB]","[Lil Nas X, Cardi B]",0.706,0.679,9.0,-5.614,1.0,0.0324,0.139,0.000070,0.465,0.657,140.081,"[rap, dance, pop]",3,";rap,dance,pop",2
42054,5MWOSsgPDm3aPvAHWghxxG,2019-06-21,day,"[7jVv8c5Fj3E9VhNjxT4snq, 4kYSro6naA4h99UJvo89HB]","[Lil Nas X, Cardi B]",0.711,0.687,9.0,-5.606,1.0,0.0318,0.138,0.000071,0.465,0.668,140.065,"[rap, dance, pop]",3,";rap,dance,pop",2


In [9]:
df= tracks_info[tracks_info['artist_name'].map(lambda x: ('Riverside' in x))]
df[df['num_artists']>1]

,track_id,release_date,release_date_precision,artist_id,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres,num_genres,genres_str,num_artists


In [10]:
artist_info_df= pd.read_csv(os.path.join('data','generated', '04_lfm_spotify_artist_info_V2.csv'), 
                         converters={"genres": literal_eval}, index_col=0)
artist_info_df= artist_info_df.dropna(axis=0)
artist_info_df= artist_info_df.reset_index(drop=True)
artist_info_df

,artist_code,artist_name,popularity,followers,genres
0,2d6NCBac0YDM7UthnFGPYz,Strumparnir,19,641.0,[pop]
1,1XVbGKrwpq5PkEbOX2z5mg,Lay Down Rotten,5,2288.0,"[german, metal]"
2,4zMX9gWf1DKhvsYHUCbBF7,T Bone Burnett,31,28844.0,[rock]
3,2eJkpFtB62y7qCx27DDNGY,Dome,10,2219.0,[british]
4,6XUNUZVWpFql1ysNCVvsNS,Daniel Tosh,42,6829.0,[]
...,...,...,...,...,...
259693,4NOgfZ4g8sFOjffbwT9qee,MAD MONSTER,5,667.0,[]
259694,3CZ6FxLtgdyxJgMufi2M8w,Adecvat_production,31,1346.0,[pop]
259695,41sqYvXOn3ATsAQ65G1zx6,María Parrado,41,52785.0,[]
259696,1JOZiF1pKROyjz9YGTCpAc,Hienpuicakiiis,0,635.0,[]


In [11]:
artists_map= artist_info_df['artist_code'].to_dict()
artists_map

{0: '2d6NCBac0YDM7UthnFGPYz',
 1: '1XVbGKrwpq5PkEbOX2z5mg',
 2: '4zMX9gWf1DKhvsYHUCbBF7',
 3: '2eJkpFtB62y7qCx27DDNGY',
 4: '6XUNUZVWpFql1ysNCVvsNS',
 5: '4kL2OPLMiNMXhojqQSNkUQ',
 6: '7tz4b7x6DwHILd1J9qj1vv',
 7: '2qc41rNTtdLK0tV3mJn2Pm',
 8: '4iPR84F8j5agZmhHwUdlPK',
 9: '5yjbUO1Jocui7RKE30zfLT',
 10: '470SGea4tIbGLTpKcYJfLx',
 11: '7kDENNLAOgnizz0fgUB6PX',
 12: '39RGo9NOqiCfbV8epXxFTj',
 13: '6rxxu32JCGDpKKMPHxnSJp',
 14: '2k7En28V8aLSsxAxE4nQdF',
 15: '7lkaHYkz51sDosEghtLxjp',
 16: '0YIhVMmaboWI25NVbZRkmo',
 17: '1DXylZlWbVvlckNqwvjTEt',
 18: '7GJMYx9DymhkSu7T75jXur',
 19: '1APhKAeROy02DedkHnbrXb',
 20: '5ndkK3dpZLKtBklKjxNQwT',
 21: '2PQ9UX4kWDE3mf2fYGRzqF',
 22: '2uTWduJySVvAw3B3KOJYxT',
 23: '5aBxFPaaGk9204ssHUvXWN',
 24: '5sQJzc7ZGwC4olbhp2kqqX',
 25: '35umDvKBwo2nG53FZTAVui',
 26: '1wUmdemiR9Dxw8A2el365A',
 27: '6nxwdNlg4g7FrLZZB43n5v',
 28: '10exVja0key0uqUkk6LJRT',
 29: '5ikCQkCt3TiuYYiLLffYL8',
 30: '046RRGSoYRQ7qIKliJJSNS',
 31: '7obdmuirClkgU94pcUy05m',
 32: '09b5OcuIELTd

In [12]:
artists_map_inv= {v:i for i,v in artists_map.items()}
artists_map_inv

{'2d6NCBac0YDM7UthnFGPYz': 0,
 '1XVbGKrwpq5PkEbOX2z5mg': 1,
 '4zMX9gWf1DKhvsYHUCbBF7': 2,
 '2eJkpFtB62y7qCx27DDNGY': 3,
 '6XUNUZVWpFql1ysNCVvsNS': 4,
 '4kL2OPLMiNMXhojqQSNkUQ': 5,
 '7tz4b7x6DwHILd1J9qj1vv': 6,
 '2qc41rNTtdLK0tV3mJn2Pm': 7,
 '4iPR84F8j5agZmhHwUdlPK': 8,
 '5yjbUO1Jocui7RKE30zfLT': 9,
 '470SGea4tIbGLTpKcYJfLx': 10,
 '7kDENNLAOgnizz0fgUB6PX': 11,
 '39RGo9NOqiCfbV8epXxFTj': 12,
 '6rxxu32JCGDpKKMPHxnSJp': 13,
 '2k7En28V8aLSsxAxE4nQdF': 14,
 '7lkaHYkz51sDosEghtLxjp': 15,
 '0YIhVMmaboWI25NVbZRkmo': 16,
 '1DXylZlWbVvlckNqwvjTEt': 17,
 '7GJMYx9DymhkSu7T75jXur': 18,
 '1APhKAeROy02DedkHnbrXb': 19,
 '5ndkK3dpZLKtBklKjxNQwT': 20,
 '2PQ9UX4kWDE3mf2fYGRzqF': 21,
 '2uTWduJySVvAw3B3KOJYxT': 22,
 '5aBxFPaaGk9204ssHUvXWN': 23,
 '5sQJzc7ZGwC4olbhp2kqqX': 24,
 '35umDvKBwo2nG53FZTAVui': 25,
 '1wUmdemiR9Dxw8A2el365A': 26,
 '6nxwdNlg4g7FrLZZB43n5v': 27,
 '10exVja0key0uqUkk6LJRT': 28,
 '5ikCQkCt3TiuYYiLLffYL8': 29,
 '046RRGSoYRQ7qIKliJJSNS': 30,
 '7obdmuirClkgU94pcUy05m': 31,
 '09b5OcuIELTdD7Ff

### Function to see examples of collaborations

In [13]:
def get_artist_name(artist_key, artist_map_):
    artist_code= artist_map_[artist_key].item()
    artist_code= artists_map[artist_code]
    artist_df= artist_info_df[artist_info_df['artist_code']==artist_code]
    return artist_df['artist_name'].values[0], artist_df['popularity'].values[0]

In [14]:
def get_collab_song_from_artists(artist1_key, artist2_key, artist_map_):
    artist1_code= artist_map_[artist1_key].item()
    artist1_code= artists_map[artist1_code]
    
    artist2_code= artist_map_[artist2_key].item()
    artist2_code= artists_map[artist2_code]
    
    df= collab_tracks[collab_tracks['artist_id'].map(lambda x: (artist1_code in x) and (artist2_code in x))]
    if not df.empty:
        return df['track_id release_date genres'.split()]
    else:
        return None

## Define model

In [15]:
def complete_snapshot(snapshot, filter_edges=True):
    data= snapshot.clone()

    data['song'].num_nodes= daily_dataset['features']['tracks.num_nodes']
    data['song'].x= daily_dataset['features']['tracks.x'].clone()
    data['song'].node_id= daily_dataset['features']['tracks.node_id'].clone()

    data['artist'].num_nodes= daily_dataset['features']['artist.num_nodes']
    data['artist'].x= daily_dataset['features']['artist.x'].clone()
    data['artist'].node_id= daily_dataset['features']['artist.node_id'].clone()

    data['genre'].num_nodes= daily_dataset['features']['genre.num_nodes']
    data['genre'].x= daily_dataset['features']['genre.x'].clone()
    data['genre'].node_id= daily_dataset['features']['genre.node_id'].clone()
    
    if filter_edges:
    
        ##*********Filter new edges
        #Filter artist_produce_song
        t= data["artist", "newproduce", "song"].edge_index.T
        artist_songs_df= pd.DataFrame(t.numpy())
        t2= data["artist", "newcollaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_songs_df= artist_songs_df[(artist_songs_df[0].isin(target_artists))]
        edge_index= torch.tensor(artist_songs_df.values).T
        data["artist", "newproduce", "song"].edge_index= edge_index

        #Filter song_has_genre
        t= data["song", "newhas", "genre"].edge_index.T
        song_genres_df= pd.DataFrame(t.numpy())
        target_songs= artist_songs_df[1].values#df2[1].unique().tolist()
        song_genres_df= song_genres_df[(song_genres_df[0].isin(target_songs))]
        edge_index= torch.tensor(song_genres_df.values).T    
        data["song", "newhas", "genre"].edge_index= edge_index

        #Filter artist_has_genre edges
        t= data["artist", "newhas", "genre"].edge_index.T
        artist_genres_df= pd.DataFrame(t.numpy())
        t2= data["artist", "newcollaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_genres_df= artist_genres_df[(artist_genres_df[0].isin(target_artists))]
        edge_index= torch.tensor(artist_genres_df.values).T
        data["artist", "newhas", "genre"].edge_index= edge_index

        ##*********Filter all edges

        #Filter artist_produce_song
        t= data["artist", "produce", "song"].edge_index.T
        artist_songs_df= pd.DataFrame(t.numpy())
        t2= data["artist", "collaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_songs_df= artist_songs_df[(artist_songs_df[0].isin(target_artists))]
        edge_index= torch.tensor(artist_songs_df.values).T
        data["artist", "produce", "song"].edge_index= edge_index

        #Filter song_has_genre
        t= data["song", "has", "genre"].edge_index.T
        song_genres_df= pd.DataFrame(t.numpy())
        song_genres_df= song_genres_df.drop_duplicates(0, keep='first')
        target_songs= artist_songs_df[1].values#df2[1].unique().tolist()
        song_genres_df= song_genres_df[(song_genres_df[0].isin(target_songs))]
        edge_index= torch.tensor(song_genres_df.values).T    
        data["song", "has", "genre"].edge_index= edge_index

        #Filter artist_has_genre edges
        t= data["artist", "has", "genre"].edge_index.T
        artist_genres_df= pd.DataFrame(t.numpy())
        t2= data["artist", "collaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_genres_df= artist_genres_df[(artist_genres_df[0].isin(target_artists))]
        edge_index= torch.tensor(artist_genres_df.values).T
        data["artist", "has", "genre"].edge_index= edge_index

        ##*********Filter reverse edges

        #Filter artist_produce_song
        t= data["song", "rev_produce", "artist"].edge_index.T
        artist_songs_df= pd.DataFrame(t.numpy())
        t2= data["artist", "rev_collaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[1].unique().tolist() + df2[1].unique().tolist())
        artist_songs_df= artist_songs_df[(artist_songs_df[1].isin(target_artists))]
        edge_index= torch.tensor(artist_songs_df.values).T
        data["song", "rev_produce", "artist"].edge_index= edge_index

        #Filter song_has_genre
        t= data["genre", "rev_has", "song"].edge_index.T
        song_genres_df= pd.DataFrame(t.numpy())
        target_songs= artist_songs_df[0].values
        song_genres_df= song_genres_df[(song_genres_df[1].isin(target_songs))]
        edge_index= torch.tensor(song_genres_df.values).T    
        data["genre", "rev_has", "song"].edge_index= edge_index

        #Filter artist_has_genre edges
        t= data["genre", "rev_has", "artist"].edge_index.T
        artist_genres_df= pd.DataFrame(t.numpy())
        t2= data["artist", "rev_collaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_genres_df= artist_genres_df[(artist_genres_df[1].isin(target_artists))]
        edge_index= torch.tensor(artist_genres_df.values).T
        data["genre", "rev_has", "artist"].edge_index= edge_index

        #********* Filter reverse new edges ***************
         #Filter artist_produce_song
        t= data["song", "rev_newproduce", "artist"].edge_index.T
        artist_songs_df= pd.DataFrame(t.numpy())
        t2= data["artist", "rev_newcollaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[1].unique().tolist() + df2[1].unique().tolist())
        artist_songs_df= artist_songs_df[(artist_songs_df[1].isin(target_artists))]
        edge_index= torch.tensor(artist_songs_df.values).T
        data["song", "rev_newproduce", "artist"].edge_index= edge_index

        #Filter song_has_genre
        t= data["genre", "rev_newhas", "song"].edge_index.T
        song_genres_df= pd.DataFrame(t.numpy())
        target_songs= artist_songs_df[0].values
        song_genres_df= song_genres_df[(song_genres_df[1].isin(target_songs))]
        edge_index= torch.tensor(song_genres_df.values).T    
        data["genre", "rev_newhas", "song"].edge_index= edge_index

        #Filter artist_has_genre edges
        t= data["genre", "rev_newhas", "artist"].edge_index.T
        artist_genres_df= pd.DataFrame(t.numpy())
        t2= data["artist", "rev_newcollaborate", "artist"].edge_index.T
        df2= pd.DataFrame(t2.numpy())
        target_artists= set(df2[0].unique().tolist() + df2[1].unique().tolist())
        artist_genres_df= artist_genres_df[(artist_genres_df[1].isin(target_artists))]
        edge_index= torch.tensor(artist_genres_df.values).T
        data["genre", "rev_newhas", "artist"].edge_index= edge_index   
    
    return data

def generate_new_edges_snapshot(snapshot):
    data= HeteroData()
    
    data['song'].num_nodes= snapshot['song'].num_nodes
    data['song'].x= snapshot['song'].x.clone()
    data['song'].node_id= snapshot['song'].node_id.clone()

    data['artist'].num_nodes= snapshot['artist'].num_nodes
    data['artist'].x= snapshot['artist'].x.clone()
    data['artist'].node_id= snapshot['artist'].node_id.clone()

    data['genre'].num_nodes= snapshot['genre'].num_nodes
    data['genre'].x= snapshot['genre'].x.clone()
    data['genre'].node_id= snapshot['genre'].node_id.clone()
    
    data["artist", "newproduce", "song"].edge_index= snapshot["artist", "newproduce", "song"].edge_index.clone()
    data["artist", "newcollaborate", "artist"].edge_index= snapshot["artist", "newcollaborate", "artist"].edge_index.clone()
    data["artist", "newhas", "genre"].edge_index= snapshot["artist", "newhas", "genre"].edge_index.clone()
    data["song", "newhas", "genre"].edge_index= snapshot["song", "newhas", "genre"].edge_index.clone()

    data["song", "rev_newproduce", "artist"].edge_index= snapshot["song", "rev_newproduce", "artist"].edge_index.clone()
    data["artist", "rev_newcollaborate", "artist"].edge_index= snapshot["artist", "rev_newcollaborate", "artist"].edge_index.clone()
    data["genre", "rev_newhas", "artist"].edge_index= snapshot["genre", "rev_newhas", "artist"].edge_index.clone()
    data["genre", "rev_newhas", "song"].edge_index= snapshot["genre", "rev_newhas", "song"].edge_index.clone()

    return data

def generate_all_edges_snapshot(snapshot):
    data= HeteroData()
    
    data['song'].num_nodes= snapshot['song'].num_nodes
    data['song'].x= snapshot['song'].x.clone()
    data['song'].node_id= snapshot['song'].node_id.clone()

    data['artist'].num_nodes= snapshot['artist'].num_nodes
    data['artist'].x= snapshot['artist'].x.clone()
    data['artist'].node_id= snapshot['artist'].node_id.clone()

    data['genre'].num_nodes= snapshot['genre'].num_nodes
    data['genre'].x= snapshot['genre'].x.clone()
    data['genre'].node_id= snapshot['genre'].node_id.clone()
    
    data["artist", "produce", "song"].edge_index= snapshot["artist", "produce", "song"].edge_index.clone()
    data["artist", "collaborate", "artist"].edge_index= snapshot["artist", "collaborate", "artist"].edge_index.clone()
    data["artist", "has", "genre"].edge_index= snapshot["artist", "has", "genre"].edge_index.clone()
    data["song", "has", "genre"].edge_index= snapshot["song", "has", "genre"].edge_index.clone()

    data["song", "rev_produce", "artist"].edge_index= snapshot["song", "rev_produce", "artist"].edge_index.clone()
    data["artist", "rev_collaborate", "artist"].edge_index= snapshot["artist", "rev_collaborate", "artist"].edge_index.clone()
    data["genre", "rev_has", "artist"].edge_index= snapshot["genre", "rev_has", "artist"].edge_index.clone()
    data["genre", "rev_has", "song"].edge_index= snapshot["genre", "rev_has", "song"].edge_index.clone()

    return data
   
    

In [16]:
"""
data=daily_dataset['edges'][0].clone()
complete_data= complete_snapshot(data)
data_time_evol= generate_new_edges_snapshot(complete_data)
data_global= generate_all_edges_snapshot(complete_data)
data_time_evol.metadata(), data_global.metadata()
"""

"\ndata=daily_dataset['edges'][0].clone()\ncomplete_data= complete_snapshot(data)\ndata_time_evol= generate_new_edges_snapshot(complete_data)\ndata_global= generate_all_edges_snapshot(complete_data)\ndata_time_evol.metadata(), data_global.metadata()\n"

In [17]:
class TimeEvolGNN(torch.nn.Module):
    def __init__(self, node_feature_dict, metadata, num_output_features):
        super(TimeEvolGNN, self).__init__()
        self.recurrent = HeteroGCLSTM(node_feature_dict, num_output_features, metadata)
    
    def forward(self, x_dict, edge_index_dict, h_dict, c_dict):
        h_0, c_0 = self.recurrent(x_dict, edge_index_dict)# h_dict, c_dict)
        h=h_0
        return h, h_0, c_0
    
class GlobalGNN(torch.nn.Module):
    def __init__(self, num_output_features):
        super(GlobalGNN, self).__init__()
        self.conv= SAGEConv((-1,-1), num_output_features)
    
    def forward(self, x_dict, edge_index_dict):
        x= self.conv(x_dict, edge_index_dict)#.relu()
        return x
    
class GlobalGNNwithAttention(torch.nn.Module):
    def __init__(self, num_output_features, n_heads=4):
        super(GlobalGNNwithAttention, self).__init__()
        self.conv= GATv2Conv((-1,-1), num_output_features, add_self_loops=False, heads=n_heads, concat=True)
        self.linear= torch.nn.Linear(num_output_features*n_heads,num_output_features)
    
    def forward(self, x_dict, edge_index_dict):
        x= self.conv(x_dict, edge_index_dict)
        return x

In [18]:
class HeteroEvolGNN3(torch.nn.Module):
    def __init__(self, node_feature_dict, metadata_time_evol, metadata_global, num_output_features):
        super(HeteroEvolGNN3, self).__init__()
        
        self.time_evol_gnn = TimeEvolGNN(node_feature_dict, metadata_time_evol, num_output_features)
        
        self.global_gnn = GlobalGNN(num_output_features)
        self.global_gnn = to_hetero(self.global_gnn, metadata_global)
        
        self.linears= ModuleDict({v:torch.nn.Linear(num_output_features,num_output_features) for v,i in node_feature_dict.items()})
        
        self.norms= ModuleDict({v:BatchNorm(num_output_features) for v,i in node_feature_dict.items()})
        
        self.global_gnn_with_att= GlobalGNNwithAttention(num_output_features)
        self.global_gnn_with_att = to_hetero(self.global_gnn_with_att, metadata_global)
        
    def forward(self, graph_evol, graph_global, h_dict=None, c_dict=None):
        
        time_emb, h, c = self.time_evol_gnn(graph_evol.x_dict, graph_evol.edge_index_dict, h_dict, c_dict)
        agg_emb = self.global_gnn(graph_global.x_dict, graph_global.edge_index_dict) 
        
        global_emb= {key: torch.add(time_emb[key],agg_emb[key]) for key, value in time_emb.items()}
        
        global_emb= {k: norm(global_emb[k]) for k, norm in self.norms.items()}
        global_emb= {k: lin(global_emb[k]) for k, lin in self.linears.items()}
        global_emb = {key: val.relu() for key, val in global_emb.items()}   

        final_emb= self.global_gnn_with_att(global_emb, graph_global.edge_index_dict) 

        return final_emb, h, c
    
class HeteroEvolGNNWithTimeOnly(torch.nn.Module):
    def __init__(self, node_feature_dict, metadata_time_evol, metadata_global, num_output_features):
        super(HeteroEvolGNNWithTimeOnly, self).__init__()
        
        self.time_evol_gnn = TimeEvolGNN(node_feature_dict, metadata_time_evol, num_output_features)
        
        self.linears= ModuleDict({v:torch.nn.Linear(num_output_features,num_output_features) for v,i in node_feature_dict.items()})
        self.norms= ModuleDict({v:BatchNorm(num_output_features) for v,i in node_feature_dict.items()})
        
        self.global_gnn_with_att= GlobalGNNwithAttention(num_output_features)
        self.global_gnn_with_att = to_hetero(self.global_gnn_with_att, metadata_global)
        
    def forward(self, graph_evol, graph_global, h_dict=None, c_dict=None):
        
        time_emb, h, c = self.time_evol_gnn(graph_evol.x_dict, graph_evol.edge_index_dict, h_dict, c_dict)
        
        global_emb= {k: norm(time_emb[k]) for k, norm in self.norms.items()}
        global_emb= {k: lin(global_emb[k]) for k, lin in self.linears.items()}
        global_emb = {key: val.relu() for key, val in global_emb.items()}   

        final_emb= self.global_gnn_with_att(global_emb, graph_global.edge_index_dict) 

        return final_emb, h, c
    
class HeteroEvolGNNWithAggOnly(torch.nn.Module):
    def __init__(self, node_feature_dict, metadata_time_evol, metadata_global, num_output_features):
        super(HeteroEvolGNNWithAggOnly, self).__init__()
                
        self.global_gnn = GlobalGNN(num_output_features)
        self.global_gnn = to_hetero(self.global_gnn, metadata_global)
        
        self.linears= ModuleDict({v:torch.nn.Linear(num_output_features,num_output_features) for v,i in node_feature_dict.items()})
        
        self.norms= ModuleDict({v:BatchNorm(num_output_features) for v,i in node_feature_dict.items()})
        
        self.global_gnn_with_att= GlobalGNNwithAttention(num_output_features)
        self.global_gnn_with_att = to_hetero(self.global_gnn_with_att, metadata_global)
        
    def forward(self, graph_evol, graph_global, h_dict=None, c_dict=None):
        
        agg_emb = self.global_gnn(graph_global.x_dict, graph_global.edge_index_dict) 
                
        global_emb= {k: norm(agg_emb[k]) for k, norm in self.norms.items()}
        global_emb= {k: lin(global_emb[k]) for k, lin in self.linears.items()}
        global_emb = {key: val.relu() for key, val in global_emb.items()}   

        final_emb= self.global_gnn_with_att(global_emb, graph_global.edge_index_dict) 

        return final_emb, None, None

In [19]:
out_channels= 320
node_feature_dict= {'song':9, 'artist':2, 'genre':1}

### Train model

In [20]:
def sample_negative_edges(positive_edges, n_nodes, n_samples):
    
    edges_= np.vstack((positive_edges[0],positive_edges[1])).T
    
    origins = []
    dest = []
    #Here we generate a negative link for each node in the snapshot
    for i in range(n_samples):
        
        o_node= np.random.randint(n_nodes)
        d_node= np.random.randint(n_nodes)

        edge_is_new= (len(edges_[(edges_[:,0]==o_node) & (edges_[:,1]==d_node)])==0)
        
        #If the (o_node, d_node) tuple already exists, try again...
        while not edge_is_new:
            d_node= np.random.randint(n_nodes)
        
            edge_is_new= (len(edges_[(edges_[:,0]==o_node) & (edges_[:,1]==d_node)])==0)

        origins.append(o_node)
        dest.append(d_node)
        
    edge_index_negs = torch.row_stack([torch.LongTensor(origins), torch.LongTensor(dest)])
    edge_label_negs= torch.zeros(edge_index_negs.shape[1]).reshape(-1,1).float()
    
    return edge_index_negs, edge_label_negs

In [21]:
def generate_node_embeddings(gnn_type, gnn_, new_edges, all_edges, h_dict=None, c_dict=None):
    
    emb_= None
    if gnn_type in 'PRESTO_full PRESTO_time_only PRESTO_agg_only'.split():
        emb_, h_dict, c_dict= gnn_(new_edges, all_edges, h_dict, c_dict)
    elif gnn_type=='time_only':
        emb_, h_dict, c_dict= gnn_(new_edges.x_dict, new_edges.edge_index_dict, h_dict, c_dict)
    elif gnn_type in 'agg_only attention_only'.split():
        emb_ = gnn_(all_edges.x_dict, all_edges.edge_index_dict)
        
    return emb_, h_dict, c_dict

In [22]:
def sample_heterograph_v2(hetero_graph, nodes_to_sample=0.15):
    num_artist_to_sample= int(len(hetero_graph['artist'].node_id)* nodes_to_sample)  
    #artists_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['artist'].node_id,num_artist_to_sample)))

    num_songs_to_sample= int(len(hetero_graph['song'].node_id)* 0.05)                                    
    songs_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['song'].node_id,num_songs_to_sample)))
 
    num_genres_to_sample= int(len(hetero_graph['genre'].node_id)*nodes_to_sample)                                    
    genres_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['genre'].node_id,num_genres_to_sample)))
    
       
    src_unique= hetero_graph.edge_index_dict[('artist', 'targetcollaborate', 'artist')][0]
    dst_unique= hetero_graph.edge_index_dict[('artist', 'targetcollaborate', 'artist')][1]
    
    artists_to_sample = torch.unique(torch.stack([src_unique, dst_unique,], dim=1))
    
    subset_dict = {
        'genre': genres_to_sample,
        'artist':artists_to_sample,
        'song': songs_to_sample,
    }
        
    data =  HeteroData()
    
    for node_type, subset in subset_dict.items():
        #print(subset)
        for key, value in hetero_graph[node_type].items():
            if key == 'num_nodes':
                if subset.dtype == torch.bool:
                    data[node_type].num_nodes = int(subset.sum())
                else:
                    data[node_type].num_nodes = subset.size(0)
            elif hetero_graph[node_type].is_node_attr(key):
                data[node_type][key] = value[subset]
            else:
                data[node_type][key] = value
   
    for edge_type in hetero_graph.edge_types:
        #print('edge_type', edge_type)
        src, _, dst = edge_type
        if src not in subset_dict or dst not in subset_dict:
            continue

     
        edge_index= hetero_graph[edge_type].edge_index
        
        new_edge_label=[]
        new_edge_index_src=[]
        new_edge_index_dst=[]
        for i in range(len(edge_index[0])):
            src_edge_index= np.where(subset_dict[src]==edge_index[0][i])[0]
            dst_edge_index= np.where(subset_dict[dst]==edge_index[1][i])[0]
            
            if (len(src_edge_index)>0) and (len(dst_edge_index)>0):
                src_edge_index= src_edge_index[0]
                dst_edge_index= dst_edge_index[0]
                new_edge_index_src.append(src_edge_index)
                new_edge_index_dst.append(dst_edge_index)
                new_edge_label.append(hetero_graph[edge_type].edge_label[i])
                
        #print(new_edge_index_src, new_edge_index_dst)
        new_edge_index = torch.stack([torch.tensor(new_edge_index_src, dtype=torch.long), 
                                      torch.tensor(new_edge_index_dst, dtype=torch.long),], dim=0)
        data[edge_type].edge_index= new_edge_index
        data[edge_type].edge_label= torch.tensor(new_edge_label).reshape(-1,1)

    return subset_dict, data

In [23]:
def sample_heterograph(hetero_graph, nodes_to_sample=0.15):
    num_artist_to_sample= int(len(hetero_graph['artist'].node_id)* nodes_to_sample)  
    artists_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['artist'].node_id,num_artist_to_sample)))

    num_songs_to_sample= int(len(hetero_graph['song'].node_id)* nodes_to_sample)                                    
    songs_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['song'].node_id,num_songs_to_sample)))
 
    num_genres_to_sample= int(len(hetero_graph['genre'].node_id)*nodes_to_sample)                                    
    genres_to_sample,_= torch.sort(torch.tensor(np.random.choice(hetero_graph['genre'].node_id,num_genres_to_sample)))
    
    subset_dict = {
        'genre': genres_to_sample,
        'artist':artists_to_sample,
        'song': songs_to_sample,
    }
    
       
    data =  HeteroData()

    for node_type, subset in subset_dict.items():
        #print(subset)
        for key, value in hetero_graph[node_type].items():
            if key == 'num_nodes':
                if subset.dtype == torch.bool:
                    data[node_type].num_nodes = int(subset.sum())
                else:
                    data[node_type].num_nodes = subset.size(0)
            elif hetero_graph[node_type].is_node_attr(key):
                data[node_type][key] = value[subset]
            else:
                data[node_type][key] = value
   
    for edge_type in hetero_graph.edge_types:
        #print('edge_type', edge_type)
        src, _, dst = edge_type
        if src not in subset_dict or dst not in subset_dict:
            continue

     
        edge_index= hetero_graph[edge_type].edge_index
        
        new_edge_label=[]
        new_edge_index_src=[]
        new_edge_index_dst=[]
        for i in range(len(edge_index[0])):
            src_edge_index= np.where(subset_dict[src]==edge_index[0][i])[0]
            dst_edge_index= np.where(subset_dict[dst]==edge_index[1][i])[0]
            
            if (len(src_edge_index)>0) and (len(dst_edge_index)>0):
                src_edge_index= src_edge_index[0]
                dst_edge_index= dst_edge_index[0]
                new_edge_index_src.append(src_edge_index)
                new_edge_index_dst.append(dst_edge_index)
                new_edge_label.append(hetero_graph[edge_type].edge_label[i])
                
        #print(new_edge_index_src, new_edge_index_dst)
        new_edge_index = torch.stack([torch.tensor(new_edge_index_src, dtype=torch.long), 
                                      torch.tensor(new_edge_index_dst, dtype=torch.long),], dim=0)
        data[edge_type].edge_index= new_edge_index
        data[edge_type].edge_label= torch.tensor(new_edge_label).reshape(-1,1)
        """    
        edge_index, _, edge_mask = bipartite_subgraph(
            (subset_dict[src], subset_dict[dst]),
            hetero_graph[edge_type].edge_index,
            relabel_nodes=True,
            size=(hetero_graph[src].node_id.max()+1, hetero_graph[dst].node_id.max()+1),
            return_edge_mask=True,
        )

        
        for key, value in hetero_graph[edge_type].items():
            if key == 'edge_index':
                data[edge_type].edge_index = edge_index
            elif hetero_graph[edge_type].is_edge_attr(key):
                data[edge_type][key] = value[edge_mask]
            else:
                data[edge_type][key] = value
        """
    return subset_dict, data

    """
    
    data = copy.copy(hetero_graph)
        
    for node_type, subset in subset_dict.items():

        if subset.dtype == torch.bool:
            num_nodes = int(subset.sum())
        else:
            num_nodes = subset.size(0)
            subset = torch.unique(subset, sorted=True)
            subset_dict[node_type] = subset

        for key, value in data[node_type].items():
            if key == 'num_nodes':
                data[node_type].num_nodes = num_nodes
            elif data[node_type].is_node_attr(key):
                data[node_type][key] = value[subset]
            else:
                data[node_type][key] = value

    for edge_type in data.edge_types:
        src, _, dst = edge_type

        src_subset = subset_dict.get(src)
        if src_subset is None:
            src_subset = torch.arange(data[src].num_nodes)
        dst_subset = subset_dict.get(dst)
        if dst_subset is None:
            dst_subset = torch.arange(data[dst].num_nodes)

        #print(edge_type, src_subset, dst_subset)
        #print(hetero_graph['artist'].num_nodes)
        edge_index, _, edge_mask = bipartite_subgraph(
            (src_subset, dst_subset),
            data[edge_type].edge_index,
            relabel_nodes=False,
            size=(src_subset.max()+1, dst_subset.max()+1),
            return_edge_mask=True,
        )

        for key, value in data[edge_type].items():
            if key == 'edge_index':
                data[edge_type].edge_index = edge_index
            elif data[edge_type].is_edge_attr(key):
                data[edge_type][key] = value[edge_mask]
            else:
                data[edge_type][key] = value

    return data
    
    """

def bipartite_subgraph(
    subset: Union[PairTensor, Tuple[List[int], List[int]]],
    edge_index: Tensor,
    edge_attr: OptTensor = None,
    relabel_nodes: bool = False,
    size: Optional[Tuple[int, int]] = None,
    return_edge_mask: bool = False,
) -> Union[Tuple[Tensor, OptTensor], Tuple[Tensor, OptTensor, OptTensor]]:

    device = edge_index.device

    src_subset, dst_subset = subset
    print('subsets', src_subset, dst_subset)
    if src_subset.dtype != torch.bool:
        src_size = int(edge_index[0].max()) + 1 if size is None else size[0]
        src_node_mask = index_to_mask(src_subset, size=src_size)
    else:
        src_size = src_subset.size(0)
        src_node_mask = src_subset

    if dst_subset.dtype != torch.bool:
        dst_size = int(edge_index[1].max()) + 1 if size is None else size[1]
        dst_node_mask = index_to_mask(dst_subset, size=dst_size)
    else:
        dst_size = dst_subset.size(0)
        dst_node_mask = dst_subset

    print("edge_index 0", edge_index)
    edge_mask = src_node_mask[edge_index[0]] & dst_node_mask[edge_index[1]]
    print('edge_mask', edge_mask)

    edge_index = edge_index[:, edge_mask]
    edge_attr = edge_attr[edge_mask] if edge_attr is not None else None
    print(edge_index)

    if edge_index.shape[1]>0:
        if relabel_nodes:
            node_idx_i = edge_index.new_zeros(src_node_mask.size(0))
            node_idx_j = edge_index.new_zeros(dst_node_mask.size(0))
            print(src_subset.shape, src_node_mask.shape, src_node_mask.sum())
            node_idx_i[src_subset] = torch.arange(int(src_node_mask.sum()),
                                                  device=node_idx_i.device)
            node_idx_j[dst_subset] = torch.arange(int(dst_node_mask.sum()),
                                                  device=node_idx_j.device)
            edge_index = torch.stack([
                node_idx_i[edge_index[0]],
                node_idx_j[edge_index[1]],
            ], dim=0)

    if return_edge_mask:
        return edge_index, edge_attr, edge_mask
    else:
        return edge_index, edge_attr

In [ ]:
sigm= torch.nn.Sigmoid()

device = 'cpu' 
n_epochs= 1#120
seq_len=12

batch_size_=512
batch_n_nodes= 180

loss_fn=  F.torch.nn.BCEWithLogitsLoss()

#model_types= 'PRESTO_full PRESTO_time_only PRESTO_agg_only time_only agg_only attention_only'.split()
model_types= 'PRESTO_full'.split()

#T= 12
#T_size=6
time_window_size= 30 #days
displayed_songs=[]

for T in [24]:#[3,6,24]:
    for T_size in [6]:
        print('\n','-'*10,f'Dataset T:{T} T_size:{T_size}','-'*10)
        graph_data_path=os.path.join('data', 'generated', 'graphs_V4', f'full_dataset_W_{time_window_size}_T_{T}_T_size_{T_size}.graph')
        daily_dataset= torch.load(graph_data_path)
        n_snapshots=len(daily_dataset['edges'].keys())
        n_iterations= len(np.arange(0,n_snapshots-13, seq_len))

        #print(f'The number of inter-snapshot batches is {n_snapshots}')
        
        data=daily_dataset['edges'][0].clone()
        complete_data= complete_snapshot(data)
        data_time_evol= generate_new_edges_snapshot(complete_data)
        data_global= generate_all_edges_snapshot(complete_data)

        for model_type in model_types:
            results=[]

            global_loss=[]
            global_f1=[]
            global_acc=[]

            global_val_f1=[]
            global_val_acc=[]

            print('\n','-'*10, model_type, '-'*10)

            for i in tqdm(np.arange(60,200, seq_len), desc='Snapshots...', leave=False):
                print('\n',"*"*10, f'Snapshot {i}-{i+seq_len}', '*'*10)
                
                init_snapshot_date= init_date_obj + relativedelta(months=i)
                final_snapshot_date= init_date_obj + relativedelta(months=i+seq_len)
                init_snapshot_date_str= datetime.strftime(init_snapshot_date,'%Y-%m-%d')
                final_snapshot_date_str= datetime.strftime(final_snapshot_date,'%Y-%m-%d')
                print('\n',"*"*2, f'Snapshot dates {init_snapshot_date_str}:{final_snapshot_date_str}', '*'*2)

                time_horizon_init= final_snapshot_date + relativedelta(months=T)
                time_horizon_final= final_snapshot_date + relativedelta(months=T+T_size)
                time_horizon_init_str= datetime.strftime(time_horizon_init,'%Y-%m-%d')
                time_horizon_final_str= datetime.strftime(time_horizon_final,'%Y-%m-%d')
                print('\n',"*"*2, f'Prediction range {time_horizon_init_str}:{time_horizon_final_str}', '*'*2)
                
                model_loss=0
                model_f1=0
                model_acc=0

                model_val_f1=0
                model_val_acc=0

                model_max_val_f1=0
                model_max_val_acc=0

                #Create model for the new snapshot
                gnn= None
                if model_type == 'PRESTO_full':
                    gnn = HeteroEvolGNN3(node_feature_dict,  data_time_evol.metadata(), data_global.metadata(), out_channels)
                elif model_type == 'PRESTO_time_only':
                    gnn = HeteroEvolGNNWithTimeOnly(node_feature_dict,  data_time_evol.metadata(), data_global.metadata(), out_channels)
                elif model_type == 'PRESTO_agg_only':
                    gnn = HeteroEvolGNNWithAggOnly(node_feature_dict,  data_time_evol.metadata(), data_global.metadata(), out_channels)
                elif model_type=='time_only':
                    gnn= TimeEvolGNN(node_feature_dict,  data_time_evol.metadata(), out_channels)
                elif model_type == 'agg_only':
                    gnn= GlobalGNN(out_channels)
                    gnn= to_hetero(gnn,data_global.metadata())    
                elif model_type=='attention_only':
                    gnn= GlobalGNNwithAttention(out_channels)
                    gnn= to_hetero(gnn,data_global.metadata())    

                gnn= gnn.train()

                optimizer = torch.optim.Adam(gnn.parameters(), lr=0.001, weight_decay=5e-4)
                gnn= gnn.to(device)
                optimizer.zero_grad()

                for epoch in tqdm(range(0,n_epochs), desc='Epochs...'):
                    epoch_loss=0
                    epoch_f1=0
                    epoch_acc=0

                    h_dict= None
                    c_dict= None  
                    batch_loss=0

                    n_samples=0
                    n_val_samples=0

                    for j in tqdm(range(i, i+seq_len), desc='Daily...', leave=False):

                        edges= daily_dataset['edges'][j]

                        print("Completing snapshot...",end="")
                        snapshot= complete_snapshot(edges, filter_edges=True) 
                        print("DONE!")
                        
                        print("Sampling data...", end="")
                        sample_dict, sample_data= sample_heterograph_v2(snapshot)
                        print("DONE!")
                        #print(sample_data)
                        #print(sample_data.edge_index_dict[('artist', 'targetcollaborate', 'artist')])
                        #e1= sample_data.edge_index_dict[('artist', 'targetcollaborate', 'artist')][0][0]
                        #e2=sample_data.edge_index_dict[('artist', 'targetcollaborate', 'artist')][1][0]
                        #print(sample_dict['artist'], sample_dict['artist'].shape)
                        #print(e1.item(), sample_dict['artist'][e1.item()])
                        #artist_name_1, artist_pop_1= get_artist_name(e1.item(), sample_dict['artist'])
                        #artist_name_2, artist_pop_2=get_artist_name(e2.item(), sample_dict['artist'])
                        #song= get_collab_song_from_artists(e1,e2, sample_dict['artist'])
                        #print(artist_name_1, artist_pop_1, artist_name_2, artist_pop_2,song)
                        
                        y_pos_edges= sample_data.edge_index_dict[('artist', 'targetcollaborate', 'artist')]
                        y_pos_labels= sample_data.edge_label_dict[('artist', 'targetcollaborate', 'artist')]  

                        if y_pos_edges.shape[1]>0:
                            n_samples= n_samples+1

                            sample_new_edges= generate_new_edges_snapshot(sample_data)
                            sample_all_edges= generate_all_edges_snapshot(sample_data)

                            emb_, h_dict, c_dict= generate_node_embeddings(model_type, 
                                                                           gnn, 
                                                                           sample_new_edges, 
                                                                           sample_all_edges, 
                                                                           h_dict, c_dict)

                            #Generate negative edges for the batch
                            n_y_edges= y_pos_edges.shape[1]
                            n_nodes= torch.max(y_pos_edges) #snapshot['artist'].num_nodes
                            y_neg_edges, y_neg_labels = sample_negative_edges(y_pos_edges, n_nodes, n_y_edges)

                            y_edges = torch.cat([y_pos_edges, y_neg_edges], dim=1)
                            y_labels = torch.cat([y_pos_labels, y_neg_labels], dim=0)

                            edge_emb_1 =  emb_["artist"][y_edges[0]]
                            edge_emb_2 =  emb_["artist"][y_edges[1]]

                            scores= (edge_emb_1 * edge_emb_2).sum(dim=-1)
                            pred=torch.reshape(scores,(-1,1))
                            loss = loss_fn(pred, y_labels)
                            epoch_loss= loss + epoch_loss

                            pred_no_grad= pred.detach()
                            pred_sig=sigm(pred_no_grad)
                            final_pred = torch.round(pred_sig)
                            
                            f1= f1_score(y_labels, final_pred)
                            acc= accuracy_score(y_labels, final_pred)

                            epoch_f1= f1 + epoch_f1
                            epoch_acc= acc + epoch_acc

                        del snapshot
                        #del sample_data

                    epoch_loss= epoch_loss / n_samples
                    model_loss= epoch_loss.detach() + model_loss

                    epoch_f1= epoch_f1 / n_samples
                    epoch_acc= epoch_acc/ n_samples
                    model_f1= epoch_f1 + model_f1
                    model_acc= epoch_acc + model_acc

                    epoch_loss.backward(retain_graph=False)
                    optimizer.step()
                    optimizer.zero_grad(set_to_none=True)

                    with torch.no_grad(): 
                        val_edges = daily_dataset['edges'][i+seq_len]
                        #print('val_edges',val_edges.edge_index_dict[('artist', 'collaborate', 'artist')][0])
                        val_snapshot= complete_snapshot(val_edges, filter_edges=False)
                        #print('val_snapshot',val_snapshot.edge_index_dict[('artist', 'collaborate', 'artist')])
                        
                        #val_sample_data= sample_heterograph(val_snapshot)
                        val_sample_dict, val_sample_data= sample_heterograph_v2(val_snapshot)

                        #print('val_sample_data',val_sample_data.edge_index_dict[('artist', 'collaborate', 'artist')])

                        val_sample_new_edges= generate_new_edges_snapshot(val_sample_data)
                        val_sample_all_edges= generate_all_edges_snapshot(val_sample_data)


                        val_emb_, h_dict, c_dict= generate_node_embeddings(model_type, 
                                                                           gnn, 
                                                                           val_sample_new_edges, 
                                                                           val_sample_all_edges, 
                                                                           h_dict, c_dict)

                        y_val_edges= val_sample_data.edge_index_dict[('artist', 'targetcollaborate', 'artist')]
                        y_val_labels= val_sample_data.edge_label_dict[('artist', 'targetcollaborate', 'artist')]  

                        #print(y_val_edges.shape)
                        if y_val_edges.shape[1]>0:
                            n_val_samples = n_val_samples +1
                            #Generate negative edges for the batch
                            n_y_val_edges= y_val_edges.shape[1]               
                            n_nodes= torch.max(y_val_edges)             
                            y_neg_val_edges, y_neg_labels = sample_negative_edges(y_val_edges, n_nodes, n_y_val_edges)

                            #print(y_val_edges[:,1])
                            #artist_1= y_val_edges[0,1].numpy().tolist()
                            #artist_2= y_val_edges[1,1].numpy().tolist()
                            
                            #artist_name_1, artist_pop_1= get_artist_name(artist_1, val_sample_dict['artist'])
                            #artist_name_2, artist_pop_2=get_artist_name(artist_2, val_sample_dict['artist'])
                            #song= get_collab_song_from_artists(artist_1,artist_2, val_sample_dict['artist'])
                            #print(artist_name_1, artist_pop_1, artist_name_2, artist_pop_2,song)

                            y_val_edges = torch.cat([y_val_edges, y_neg_val_edges], dim=1)
                            y_val_labels = torch.cat([y_val_labels, y_neg_labels], dim=0)

                            val_edge_emb_1 =  val_emb_["artist"][y_val_edges[0]]
                            val_edge_emb_2 =  val_emb_["artist"][y_val_edges[1]]

                            val_scores= (val_edge_emb_1 * val_edge_emb_2).sum(dim=-1)
                            val_pred=torch.reshape(val_scores,(-1,1))
                            val_pred_sig=sigm(val_pred)
                            final_val_pred = torch.round(val_pred_sig)
                            
                            #Logic to show examples of collaboratons
                            l2= final_val_pred[y_val_labels >0]
                            l22= l2[l2>0]

                            e1= y_val_edges[0][y_val_labels.flatten() >0]
                            #print(e1.shape, l2.shape)
                            e1= e1[l2>0]
                            e2= y_val_edges[1][y_val_labels.flatten() >0]
                            e2= e2[l2>0]
                            #print(l22,e1,e2)
                            #print("*"*15)

                            for a_index in range(len(e1)):
                                a1_key= e1[a_index].numpy().tolist()
                                a2_key= e2[a_index].numpy().tolist()
                                artist_name_1, artist_pop_1= get_artist_name(a1_key, val_sample_dict['artist'])
                                artist_name_2, artist_pop_2=get_artist_name(a2_key, val_sample_dict['artist'])

                                if artist_pop_1 > 70 or artist_pop_2 > 70:
                                    song_lst= get_collab_song_from_artists(a1_key,a2_key, val_sample_dict['artist'])
                                    #print(a1_key, artist_name_1, artist_pop_1, a2_key, artist_name_2, artist_pop_2)
                                    if song_lst is not None:
                                        #print(song_lst)
                                        for s_index, s in song_lst.iterrows():
                                            track_to_show_id=s['track_id']
                                            release_date= s['release_date']
                                            #print(track_to_show_id, release_date)
                                            if (not track_to_show_id in displayed_songs) and (release_date >= time_horizon_init) and (release_date <= time_horizon_final):
                                                print(artist_name_1, artist_pop_1, artist_name_2, artist_pop_2, s)
                                                print("*"*10)
                                                displayed_songs.append(track_to_show_id)
                                                break

                            val_acc= accuracy_score(y_val_labels, final_val_pred)
                            val_f1= f1_score(y_val_labels, final_val_pred)

                            if val_f1 > model_val_f1:
                                model_val_f1= val_f1

                            if val_acc > model_val_acc:
                                model_val_acc= val_acc


                        #del val_loader
                        del val_snapshot
                        del val_sample_data


                    #print(f"--Epoch: {epoch:03d}, Loss: {epoch_loss:.4f}, F1: {epoch_f1:.4f}, ACC: {epoch_acc:.4f}, val-F1: {val_epoch_f1:.4f}, val-ACC: {val_epoch_acc:.4f}")

                model_f1= model_f1 / n_epochs
                model_acc= model_acc / n_epochs
                model_loss= model_loss / n_epochs

                print(f"-Model: {i:03d}, Loss: {model_loss:.4f}, F1: {model_f1:.4f}, ACC: {model_acc:.4f}, val-F1: {model_val_f1:.4f}, val-ACC: {model_val_acc:.4f}")

                global_f1.append(model_f1)
                global_loss.append(model_loss)
                global_acc.append(model_acc)

                global_val_f1.append(model_val_f1)
                global_val_acc.append(model_val_acc)

                del gnn
                del optimizer

            global_f1_mean= np.mean(global_f1)
            global_acc_mean= np.mean(global_acc)
            global_loss_mean= np.mean(global_loss)
            global_val_f1_mean= np.mean(global_val_f1)
            global_val_acc_mean= np.mean(global_val_acc)    

            global_f1_std= np.std(global_f1)
            global_acc_std= np.std(global_acc)
            global_loss_std= np.std(global_loss)
            global_val_f1_std= np.std(global_val_f1)
            global_val_acc_std= np.std(global_val_acc)  

            results.append((model_type, T, T_size, global_loss_mean, global_f1_mean, global_acc_mean,global_val_f1_mean, global_val_acc_mean, global_loss_std, global_f1_std, global_acc_std, global_val_f1_std, global_val_acc_std))
            results_df= pd.DataFrame(results, columns='model_type T T_size train_loss_mean train_f1_mean train_acc_mean val_f1_mean val_acc_mean train_loss_std train_f1_std train_acc_std val_f1_std val_acc_std'.split())
            results_df.to_csv(os.path.join('results', f'results_type_{model_type}_T_{T}_T_size_{T_size}.csv'))

            print(f"Global: {i:03d}, Loss: {global_loss_mean:.4f}, F1: {global_f1_mean:.4f}, ACC: {global_acc_mean:.4f}, val-F1: {global_val_f1_mean:.4f}, val-ACC: {global_val_acc_mean:.4f}")


 ---------- Dataset T:24 T_size:6 ----------

 ---------- PRESTO_full ----------


Snapshots...:   0%|          | 0/12 [00:00<?, ?it/s]


 ********** Snapshot 60-72 **********

 ** Snapshot dates 2009-01-01:2010-01-01 **

 ** Prediction range 2012-01-01:2012-07-01 **


Epochs...:   0%|          | 0/3 [00:00<?, ?it/s]

Daily...:   0%|          | 0/12 [00:00<?, ?it/s]

...

Pyotr Ilyich Tchaikovsky 71 Tbilisi Symphony Orchestra 29 track_id        02djPerx76u9hkJRhv6zFU
release_date       2012-01-01 00:00:00
genres                [classical, pop]
Name: 1554318, dtype: object
**********
Jamey Johnson 65 George Strait 75 track_id          4Mu056Wyd7GOZQK9OMq9dG
release_date         2012-01-01 00:00:00
genres          [country, rock, country]
Name: 1557294, dtype: object
**********
The Game 75 J. Cole 88 track_id                                   243C7wnj16KiWFlcIXwKf3
release_date                                  2012-01-01 00:00:00
genres          [hip-hop, rap, trap, pop, hip-hop, rap, indie,...
Name: 1568149, dtype: object
**********
Mac Miller 86 Joey Bada$$ 76 track_id                       50wicJjTmQSkZ6XluATpCi
release_date                      2012-03-23 00:00:00
genres          [hip-hop, rap, hip-hop, hip-hop, rap]
Name: 1569716, dtype: object
**********
Future 92 Tasha Catour 22 track_id        0NrK4sqr5wcvgSIaYcxSID
release_date       2012-01-17 0

Rihanna 95 Hector Fonseca 27 track_id        3A6iniWiW222psfVuGwE7F
release_date       2012-05-25 00:00:00
genres                    [pop, house]
Name: 1689690, dtype: object
**********
Ali B 53 Akon 83 track_id                          5LsRsya93LrF5GQtQMNZYG
release_date                         2012-04-23 00:00:00
genres          [hip-hop, pop, dance, pop, hip-hop, pop]
Name: 1697008, dtype: object
**********
Florence + The Machine 77 Maya Jane Coles 50 track_id                  2k7mlYWt6GLMSwX4JbEOsu
release_date                 2012-01-01 00:00:00
genres          [alternative, pop, disco, house]
Name: 1709299, dtype: object
**********
Zedd 77 Matthew Koma 60 track_id                                   4xvoF4CaBK2SXrq40KqPdx
release_date                                  2012-01-01 00:00:00
genres          [pop, edm, rap, german, dance, house, electro,...
Name: 1709400, dtype: object
**********
Hampenberg 26 Pitbull 85 track_id                                   6zBFMhKoH0FxbXjclF5Uli
r

Dry 48 GIMS 74 track_id                                   0BU5e8rDml803bp7mbouC0
release_date                                  2012-02-19 00:00:00
genres          [hip-hop, french, pop, hip-hop, rap, french, pop]
Name: 1804095, dtype: object
**********
Ne-Yo 81 Bob Sinclar 70 track_id                                   7qBNjRElxOnNlf1xAn5DIZ
release_date                                  2012-01-01 00:00:00
genres          [dance, pop, pop, edm, dance, house, disco, el...
Name: 1805348, dtype: object
**********
Jennifer Lopez 78 Flo Rida 81 track_id                                  5fuIfc73negWUuLikBIQTR
release_date                                 2012-06-22 00:00:00
genres          [pop, edm, rap, dance, hip-hop, rap, dance, pop]
Name: 681762, dtype: object
**********
Jennifer Lopez 78 Michael Woods 31 track_id                                  0LHlyOiXtZLWUJ2njUxvYA
release_date                                 2012-01-01 00:00:00
genres          [rap, dance, pop, pop, edm, rap, dance, 

Antonio Vivaldi 73 Rachel Podger 41 track_id        1r6XbFa7zsCESrgMj8QsGQ
release_date       2012-06-01 00:00:00
genres          [classical, classical]
Name: 1902396, dtype: object
**********
Pitbull 85 Vedo-No Shake 4 track_id            3Vd77qGGNUs4bZ7c5JTObC
release_date           2012-05-08 00:00:00
genres          [hip-hop, rap, dance, pop]
Name: 1904943, dtype: object
**********
Eddie Vedder 64 Jack Johnson 78 track_id                             0J1iFISejodqBIy0Q9AS0N
release_date                            2012-01-01 00:00:00
genres          [rock, acoustic, neo mellow, acoustic, pop]
Name: 1911681, dtype: object
**********
Damian Marley 68 Jack Johnson 78 track_id                3WxLFkLQUe7QXzTpZVU8zQ
release_date               2012-01-01 00:00:00
genres          [reggae, neo mellow, jawaiian]
Name: 1918691, dtype: object
**********
The Alchemist 72 Willie The Kid Master 3 track_id             3rLc8twhnuZKNBNEoh48go
release_date            2012-06-11 00:00:00
genres          

Daily...:   0%|          | 0/12 [00:00<?, ?it/s]

--------------------------------------------------------

In [ ]:
print("That' all folks!")